# Download and Extract Data

In [ ]:
![ -d "Useful-Corpora-for-Text-Mining-in-Persian-Language" ] || git clone "https://github.com/Text-Mining/Useful-Corpora-for-Text-Mining-in-Persian-Language"

Cloning into 'Useful-Corpora-for-Text-Mining-in-Persian-Language'...
remote: Enumerating objects: 18, done.
remote: Total 18 (delta 0), reused 0 (delta 0), pack-reused 18 (from 1)
Receiving objects: 100% (18/18), 478.78 MiB | 24.46 MiB/s, done.
Updating files: 100% (8/8), done.


In [ ]:
!pip install -q unrar
![ -f farsnews.json ] || unrar x '/content/Useful-Corpora-for-Text-Mining-in-Persian-Language/News/FarsNews 97/farsnews.part01.rar'


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/Useful-Corpora-for-Text-Mining-in-Persian-Language/News/FarsNews 97/farsnews.part01.rar

Extracting  farsnews.json                                                  0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38%

Extracting from /content/Useful-Corpora-for-Text-Mining-in-Persian-Language/News/FarsNews 97/farsnews.part02.rar

...         farsnews.json                                                 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67%

#Read Data

In [ ]:
import json
counter = 0
news=[] # { CategoryPanel[], NewsDate, NewsTitle, NewsSummary, NewsBody, GetComments: { CommentsJsonArray[] } }

# Read just one line of data for now
for line in open('farsnews.json','r', encoding='utf-8-sig'):
  news.append(json.loads(line))
  counter+=1
  if(counter == 1):
    break

# Dadma Module


## Modules Installation

In [ ]:
# dadmatools installation
!pip install dadmatools
# Docs: https://github.com/Dadmatech/DadmaTools

## Imports

In [ ]:
from dadmatools.normalizer import Normalizer as DadmaNormalizer

# Pipeline
# Containing Tokenizer, Lemmatizer, POS Tagger, Dependancy Parser, Constituency Parser, Kasreh, spellcheker.
import dadmatools.pipeline.language as language

## Process Data With Dadma

In [6]:
def to_list(lists):
  r = []
  for x in lists:
    r.append(x['text'])
  return r

In [5]:
# Clean and standardize the text
def dadma_normalize(text):
  # preprocesing => remove '\n'
  removeNewline = text.replace("\r\n", "")

  normalizer = DadmaNormalizer(
    # full_cleaning=True,
    # unify_chars=True,
    # refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=True,
    # remove_html=False,
    # remove_stop_word=False,
    # replace_email_with="<EMAIL>",
    # replace_number_with=None,
    # replace_url_with="",
    # replace_mobile_number_with=None,
    # replace_emoji_with=None,
    # replace_home_number_with=None
  )
  normalized = normalizer.normalize(removeNewline)

  print("(DADMA-TOOLS): normalized body: ", normalized)
  return normalized

In [7]:
# Splits the text into individual words or sentences
def dadma_tokenize(text):

  # here lemmatizer and pos tagger will be loaded
  # as tokenizer is the default tool, it will be loaded as well even without calling
  # pips = 'tok,lem,pos,dep,chunk,cons,spellchecker,kasreh'

  pips = 'tok' #we need tokenizer
  nlp = language.Pipeline(pips)

  # you can see the pipeline with this code
  # print(nlp.analyze_pipes(pretty=True))

  # tokenized_text is an SpaCy object
  tokenized_text = nlp(text)
  token_list = to_list(language.to_json(pips, tokenized_text)[0])

  print("(DADMA-TOOLS): tokenized body: ", token_list)
  return token_list

In [11]:
# Reduce words to their root or stem form
def dadma_stem(text):
  # pips = 'tok,lem,pos,dep,chunk,cons,spellchecker,kasreh'
  pips = 'tok,lem'
  nlp = language.Pipeline(pips)

  results = language.to_json(pips, nlp(text))
  stemmed = []
  for word in results[0]:
    stemmed.append(word['lemma'])

  print("(DADMA-TOOLS): stemmer result: ", stemmed)

In [9]:
# Take a news article as input and apply the steps to its body text
def dadma_process_news(news):
  news_body = news['NewsBody']
  normalized_body = dadma_normalize(news_body)
  tokenized_body = dadma_tokenize(normalized_body)
  stemmed_body = dadma_stem(normalized_body)

## Results

In [ ]:
for i in range(counter):
  print(f"Text: {news[i]['NewsBody']}\n")

  print("dadmatools Proccess: ")
  dadma_process_news(news[i])